# **Data-Driven Analysis of Hacken Lee's Setlist for **

## 1. Introduction

### 1.1 Project Goals
Analyze the track selection from three Hacken Lee's concerts that share the same theme. 



Specifically, this project explores:
- *How are hit songs and lesser-known sidetracks balanced across the setlists?*
- *Which albums are most favoured?*
- *How do the setlists vary across different concert locations?*

### 1.2 Why Hacken Lee?
First of all, I like him. 

Secondly, Hacken Lee is a prominent Cantopop singer known for his rich discography: he is one of the few singers whose career spans from the 1980s to the 2020s, maintaining both popularity and a steady output of albums. He is also among the rare artists who received music awards in Hong Kong across every decade from the 1980s to the 2010s. 

For established singers like Hacken, with a vast pool of possible songs to perform, crafting a setlist often means striking a balance between hit songs and lesser-known sidetracks. This characteristic makes his concerts especially interesting for setlist analysis.



### 1.3 Why These Three Concerts?

In May 2023, Hacken Lee collaborated once again with the Hong Kong Philharmonic Orchestra for a concert series at the Hong Kong Coliseum. Following that, he brought the concert on tour to mainland China in 2024 with the Guangzhou Symphony Orchestra. Earlier this year (2025), he teamed up with the Macao Orchestra for the third time to present the highly acclaimed concert of the same theme in Macau.

Although the concert title varied slightly at each location—**Hacken Lee x HK Phil 2023** (弦續 李克勤·港樂演唱會) in Hong Kong, **弦续 李克勤巡回演唱会** in mainland China, and **Hacken Lee Symphonic Live in Londoner with Macao Orchestra** (李克勤·我們的交響樂) in Macau—the central themes remained consistent: *continuation* and *collaboration* 

- *Continuation* as  by the title (also the theme song) 「弦續」 (a homophone of 「延續」 in Cantonese), symbolizing the continuation of music
-  *Collaboration* is embodied in the fusion with live orchestras

The three setlists chosen are from:
- **2023/05/20 @ Hong Kong**  (data sourced from *HackenZone*@FB)
- **2024/11/30 @ Guangzhou**  (firsthand experience)
- **2025/02/09 @ Macau**         (also firsthand)

These represent the finale concerts for the Hong Kong leg, the mainland China tour, and the Macau shows respectively. *(Although the Macau concert wasn't actually the final one—as Hacken will be returned to perform there later this year—I chose it under that assumption while working on this project.)*

The encore section of a live concert is especially critical in leaving a lasting impression. The finale concert usually has the most electrifying atmosphere, and performers often sing more songs during the encore. For example, Hacken performed **14 encore songs** at the finale of his *30th Anniversary Concert* in 2017.

Another reason for choosing these three concerts is that Cantonese is the dominant spoken language in all three cities. As Hacken has released albums in both Cantonese and Mandarin, he often adapts his setlists based on the audience’s language. In non-Cantonese-speaking areas, some tracks are swapped out. The consistency in language across these three locations allows for a more cohesive and meaningful comparison of setlist design.



## 2. Retrieving information from Spotify
### 2.1 Setup & Authorization
There is a Python library for the Spotify Web API called [Spotipy](https://spotipy.readthedocs.io/en/2.25.1/), which can be used for retrieving music data provided by the Spotify platform. For <b>authorization</b>, I followed the step on <em>Spotipy</em> documentation page and set environment variables.

In [2]:
import pandas as pd
import spotipy
import time
import requests
from spotipy.oauth2 import SpotifyClientCredentials

#spo = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
path = "dataset/"

### 2.2 Retrieving overall album data
The first step is to grab the whole album list for the given artist `id` using `artist_albums()`.

NOTE1: The parameter `include_groups` have four valid values: `album`, `single`, `appears_on` and `compilation`. For the purpose of this project, there is no need to retrieve information `appears_on` type since it is mostly consisted of Spotify's self-made set of tracks, rather than those officially released by record companies.

NOTE2: `artist_albums()` has a maximum number (**50**) of items to return. The while loop is needed if the artist has over 50 albums.

`pandas` library is then used to turn the results into a dataframe object, drop unnecessary columns, and save the file locally.



In [ ]:
albums = []
hacken = spo.artist("https://open.spotify.com/artist/3PV11RNUoGfX9tMN2wVljB")

album = spo.artist_albums(hacken['id'], include_groups='album',limit=50)
single = spo.artist_albums(hacken['id'], include_groups='single',limit=50)
compilation = spo.artist_albums(hacken['id'], include_groups='compilation',limit=50)
albums.extend(album['items'])
albums.extend(single['items'])
albums.extend(compilation['items'])

while album['next']:
    album = spo.next(album)
    albums.extend(album['items'])
while single['next']:
    single = spo.next(single)
    albums.extend(single['items'])
while compilation['next']:
    compilation = spo.next(compilation)
    albums.extend(compilation['items'])

df = pd.DataFrame(data=albums,columns=['name','id','release_date','album_type'])
df.to_excel(path+"Hacken_Lee_Albums_Full.xlsx",index=False)

This will yield a overall album dataset of **4** columns(`name`, `id`, `release_date`, `album_type`)
![](img/ex0.png)

In [ ]:
ex_overall = pd.read_excel(path+"Hacken_Lee_Albums_Full.xlsx")
ex_overall.head()

### 2.3 Retrieving track data for each album
For each album, `album_tracks()` is used to get every track information inside that album. Only the `id` and `name` of each track data are necessary for the next step, once again `pandas` is used.

NOTE: `time.sleep()` is extremely needed in this step, otherwise the process would be cutoff easily.


In [ ]:
for i in range(len(albums['id'])):
    print(albums['name'][i]," started")
   
    results = spo.album_tracks(albums['id'][i])
    tracks = results['items']
    while results['next']:
        results = spo.next(results)
        tracks.extend(results['items'])
        
    tracks_df = pd.DataFrame(data=tracks,columns=['name','id'])
    print(albums['name'][i]," dataframe generated")
    tracks_df["album_id"] = albums['id'][i]
    tracks_df.to_excel(path+str(albums['name'][i])+".xlsx",index=False)
    print(str(albums['name'][i])," finished")
    time.sleep(30) 

At this stage, a separate dataset is created for each album, contianing **3** columns:
- `name` (track title)
- `id` (track unique identifier)
- `album_id` (parent album id)

![](img/ex1.png)

In [ ]:
ex_album = pd.read_excel(path+"李克勤慶祝成立30週年演唱會.xlsx")
ex_album.head()

### 2.4 Retrieving audio features for each track
Here comes the main part of data extraction. One of the most valuable asset of Spotify's database is the audio features of each track. The function `audio_features()` supports a list of `id` as parameter, which simplifies the process -- instead of retrieving audio features of a single track every time, passing as a list reduces the number of calling `audio_features()` and avoiding possible timeout. 

`Pandas` is once again used to drop unnecessary columns and save the file.

NOTE1: please handle the parameter for `time.sleep()` in this step carefully

NOTE2: the information provided by `audio_features()` doesn't include the name of the album and the name of track, to add these:
- `features_df['album_name'] = albums['name'][i]` *to copy cell value at the ith row, name column from the overall album dataset* (`album_name`)
- `features_df.loc[:,'track_name'] = album['name']` *to copy the whole name column from the current album dataset* (`track_name`)

In [ ]:
for i in range(len(albums['id'])):
 
    if (i>20 and i%20==0):
        time.sleep(5)
    print(albums['name'][i]+" started")
   
    album = pd.read_excel(path+albums['name'][i]+'.xlsx') 
 
    track_ids = []
    for j in range(len(album['id'])):
        track_ids.append(album['id'][j])
    time.sleep(1)
    print("Track ID concatenated")
    results = spo.audio_features(track_ids)
    
    features_df = pd.DataFrame(data=results,columns=results[0].keys())
    print("DataFrame Generated")
    time.sleep(1)
    features_df = features_df.drop(columns=['type','uri','track_href','analysis_url'])
 
    features_df['album_name'] = albums['name'][i]
    features_df.loc[:,'track_name'] = album['name']
    features_df.to_excel(path+albums['name'][i]+'_features.xlsx',index=False)
    
    print(albums['name'][i]+" finished")
    print('time to sleep')
    time.sleep(25)

Similar to the last step, this will generate a  dataframe that have **14** columns for every album. Below is an example
![](img/ex2.png)

In [ ]:
ex_album_feature = pd.read_excel(path+"李克勤慶祝成立30週年演唱會_features.xlsx")
ex_album_feature.head()

### 2.5 Concatenating the feature datasets as one dataset
For the purpose of searching and extracting one specific track information, concatenating the feature datasets into one overall dataset would be extremely useful in terms of saving time. 

NOTE: For readability's sake, this line `album = album.iloc[:, [15, 11, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13]] ` is used to reorder the columns, so that `track_name`, `id`, `album_name` would be the first three columns


In [ ]:
album_features = []
total_albums_features = []  
for i in range(len(albums['id'])):
    album = pd.read_excel(path + albums['name'][i] + '_features.xlsx')
    album = album.iloc[:, [15, 11, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13]] 
    albums.append(album)

album_features_total = pd.concat(total_albums_features)
album_features_total.head()

### 2.6 Retrieving the PlayCount data using Public API
**Disclaimer**: This project is to be used for **education purpose** only.

Let's back to this project's first research question: <em>How are hit songs and lesser-known sidetracks balanced across the setlists?</em>. To answer this, we need to define what are hit songs and what are sidetracks. Every fan may give a different opinion about this. Thus, an objective measure is needed. 

#### 2.6.1 Popularity data from Spotify and why not applicable
Spotify's API provides the popularity data of a track object, which <em>"is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are"</em>. In some cases it may reflect how popular the track is, but its nature in which duplicate tracks are rated independently determined that it's not suitable for this project.

Why? It's not unusual that record companies release compilation albums every few years and listeners may tend to listen one compilation album rather than search for the particular album when one song is released in. Therefore, it's not really a proper choice as one track's popularity data would be dispersed into different version.

#### 2.6.2 PlayCount data using Public API
Spotify also have the play counts for each track, although not accessible using its official API. After doing some research, I found out that for the same track, Spotify adds up every version's play counts and treat it the same for every version -- which is exactly what I need. 

A sincere thank to the author of [this github project](https://github.com/entriphy/sp-playcount-librespot), who provides a public api to retrieve the play count of all tracks in a Spotify album.

`requests` library is used to handle the response returned, `pandas` is then used to merge the results to previous features dataset.



In [ ]:
hacken_albums = pd.read_excel(path+"Hacken_Lee_Albums_Full.xlsx")

base_url = "https://api.t4ils.dev/albumPlayCount"
for i in range(len(hacken_albums['id'])):
    if (i>61 and i%10==0):
        print("it's time to sleep")
        time.sleep(5)
    album_id = hacken_albums['id'][i]  

  
    url = f"{base_url}?albumid={album_id}"
    response = requests.get(url)
    print(hacken_albums['name'][i]," started")

    if response.status_code == 200:
        data = response.json()
    
        if data.get('success'):
            play_count_data = []
            album_data = data.get('data',{})
            for disc in album_data.get('discs', []):
                for track in disc.get('tracks', []):
                    track_id = track.get('uri').split(":")[-1] 
                    play_count = track.get('playcount')
                    play_count_data.append({'id': track_id, 'play_count': play_count})
            
         
            play_count_df = pd.DataFrame(play_count_data)
            print(play_count_df.head())
          
            existing_data = pd.read_excel(path+hacken_albums['name'][i]+"_features.xlsx")
            updated_data = existing_data.merge(play_count_df, on='id', how='left')
        
            updated_data.to_excel(path+hacken_albums['name'][i]+"_features.xlsx", index=False)
            print("Play count data successfully merged into the Excel sheet.")
        
        else:
            print("API response indicates failure.")
    else:
        print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")
    time.sleep(2)

#### 2.6.3 Update the overall feature dataset
After grabbing the play count data for each album, the overall dataset needs to be updated as follows. 

NOTE: The order of albums in the overall feature dataset follows the same order as in the overall album dataset, which simplifies the process -- just a nested for loop will do the trick.

In [ ]:
playcount_list = []

for i in range(len(hacken_albums['id'])):
    album_name = hacken_albums.loc[i, "name"]
    print(f"{album_name} started")

    album = pd.read_excel(path + album_name + "_features.xlsx")
  
    for j in range(len(album['id'])):
        playcount_list.append(album.loc[j, 'play_count'])

hacken_features = pd.read_excel(path + "Hacken_Lee_Albums_Features_Full.xlsx")
hacken_features['play_count'] = playcount_list

hacken_features.to_excel(path+"Hacken_Lee_Albums_Features_Updated.xlsx",index=False)

Till now, everything needed from Spotify has been retrieved and saved in several datasets. Let's proceed to extract setlist data.

## 3. Extracting Setlist Data
NOTE: In this part, only two overall datasets are needed:
- `hacken_albums`: the overall album list which contains every album's `name`, `id`, `release_date` and `album_type`
- `hacken_features`: the overall feature list which contains every track's audio features data, album_name, track_name, and play_count

### 3.1 Searching for specific track data
The first step is obviously to make a list of names of the tracks that Hacken performed in those three concerts respectively. Using `pandas`'s function `.isin()` can do a quick and shallow filter based on the tracks' name. But a problem emerges as follows: how to deal with duplicates? Pick my favourite song as an example, here is the result for `hacken_features[hacken_features['track_name'].str.contains("好戲之人")][["track_name","album_name","play_count"]]`.
![](img/ex3.png)

It's clearly that one single track would have several versions (studio album, compilation, or live). In order to answer the second research question: *Which albums are most favoured?*, only one version is needed, that is, the first album that the track is released in. 

**NOTE**:
1. Since the feature dataset doesn't contain information of release date, a merge operation is required. The logic of this merge step is to add the `release_date` cell value based on the match for album name. As the representative column of album name have different names (`album_name` in feature dataset and `name` in album dataset), an *inner join* is the only method that works.

2. The type of `release_date` is string, it needs to be reformated to datetime object first. 

Below is an example code for extracting the data for setlist in Macau concert.

In [ ]:
track_list_macau = [
    "希望", "一個都不能少", "破曉時份", "只想你會意", "舊歡如夢", 
    "護花使者", "藍月亮", "C3PO", "回首", "一生不變", 
    "Victory", "天水、圍城", "刻不容緩", "最愛", "沒有你贏了世界又如何","告別校園時",
    "再見演奏廳","深深深","紅日","夏日之神話","月半小夜曲","我不會唱歌","高妹","合久必分","飛花"
]


filtered_df = hacken_features[hacken_features['track_name'].isin(track_list_macau)]


merged_filtered_df = pd.merge(
    filtered_df, 
    hacken_albums[['name', 'release_date']], 
    left_on='album_name', 
    right_on='name', 
    how='inner'
)


merged_filtered_df['release_date'] = pd.to_datetime(merged_filtered_df['release_date'])

plot_data = merged_filtered_df.loc[merged_filtered_df.groupby('track_name')['release_date'].idxmin()]

plot_data['release_year'] = plot_data['release_date'].dt.year

plot_data = plot_data.drop(columns=['release_date',"name"])

### 3.2 Manual Adjustment
#### 3.2.1 Correction for album name

Due to copyright issues, some of Hacken's albums are not available on Spotify. (Particularly those released by Impact Entertainment.) Thus, in the last step of selecting the first release album, what is selected is the earliest album that Spotify has data for. To pursue authenticity, manual correction for albums is necessary in this step.

#### 3.2.2 Adding record company information

For visualization, I find it would be more intuitive if every track/album is labelled with a color corresponding to the record company.

As said in the introduction, Hacken's career spans from the 1980s to 2020s (and is still ongoing), *unavoidably* (I use this word in considering the fact of record industry development in Hong Kong from the 1980s to now) he has collaborated with 5 different record companies:
- PolyGram (*1986-1993*)
- Star (*1993-1996*)
- Music Impact Entertainment (*1996-1998*)
- Universal Music Hong Kong (*1999-2016*)
- Emperor Entertainment Group (*2016-now*)

Although Spotify contains a copyright statement for an album, for some albums the information is missing. In this case, it would be more time-saving to enter the record company value manually as I have prior knowledge (with the help of those physical CDs I owned).

#### 3.2.3 Classifying tracks based on position
Normally a setlist of a concert is consisted of fixed songs and encore songs. Classifying the tracks into `main` and `encore` types allows me to have a better examine at the first research question.

**NOTE**
1. Usually the encore part lies the last of a concert, but here the setlist of the Hong Kong one makes the exception -- as Hacken performed different songs every night during the orchestra's intermission time. Hence I feel more appropriate to put those four songs into the category of '*encore songs*'.



In [ ]:
hk_setlist = pd.read_excel(path+"23HongKong.xlsx")
hk_setlist.head()

## 4. Visualization
### 4.1 Quadrant Analysis
Similar to a scatter plot, a quadrant chart is used to identify patterns or trends in data using an XY axis. However, what distinguishes a quadrant chart is that the origin of the two axes lies at the middle point, typically the center of the field, thus forming four distinct areas. The key difference between these two types of charts is the direction of values increases -- in quadrant charts, values increase in positive and negative directions as they expand from the center. 

To address the first question: *How are hit songs and lesser-known sidetracks balanced across the setlists?* The play count data serves as one axis, as it was established in **Section 2.6** as the objective classification metric. A straightforward approach is to use the average play count as a threshold: tracks exceeding this value are classified as "hit songs", while other are considered sidetracks. 

For the other axis, the track's release year is selected. The year range is deliberately fixed from *1985* to *2025* (rather than using the actual earliest and latest years) for two reasons:
1. A fixed range facilitates cross-comparison among the three setlists
2. The middle point, or the *origin*, corresponds to 2005 -- the year Hacken won *Most popular male signer* for the third time


Thus, this step aims to create quadrant charts with:
- X-axis: Release year (from *1985* to *2025*)
- Y-axis: Log-transformed play count (from *8* to *18*)

**Key Notes:**
1. Log transformation is applied to play counts due to their extreme range (e.g., a ~1 million difference between the top two tracks). Without this, most tracks would cluster near the bottom.
2. Color-coding track spots by their record company (via the `record_company` column) may reveal *label-specific* patterns, also color-coding track text by the type (via the `rundown` column) mentioned in **Section 3.2.3** may reveal *position-specific* patterns.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib
import matplotlib.font_manager as fm


matplotlib.rcParams['font.family'] = ['Heiti TC']

no_duplicate_df = pd.read_excel(path+"23HongKong.xlsx")


no_duplicate_df['log_play_count'] = np.log1p(no_duplicate_df['play_count'])

color_id = {
     '寶麗金': '#84594d',  
    '星光': '#a9d8e4',  
    '藝能': '#c6c6c6',  
    '環球': '#a7dc91',  
    '英皇': '#3975b1'   
}
tran_Name = {
    '寶麗金': 'PolyGram', 
    '星光': 'Star',  
    '藝能': 'Music Impact Entertainment',
    '環球': 'Universal', 
    '英皇': 'EEG'  
}


no_duplicate_df['RecordCompanyEN'] = no_duplicate_df['record_company'].map(tran_Name)
no_duplicate_df['color'] = no_duplicate_df['record_company'].map(color_id)
rundown_color_map = {'main': 'black', 'encore': '#c9a1ed'}
no_duplicate_df['text_color'] = no_duplicate_df['rundown'].map(rundown_color_map)

x = no_duplicate_df['release_year']
y = no_duplicate_df['log_play_count']


x_min, x_max = 1985, 2025
y_min, y_max = 8,18
x_mid = x_min + (x_max - x_min) / 2
y_mid = y_min + (y_max - y_min) / 2


fig, ax = plt.subplots(figsize=(12, 8))

for company, group in no_duplicate_df.groupby('RecordCompanyEN'):
    ax.scatter(
        group['release_year'],
        group['log_play_count'],
        label=company,
        color=group['color'].iloc[0],
        edgecolor='black',
        s=100
        
    )

texts = []
for i, row in no_duplicate_df.iterrows():
    texts.append(
        ax.text(
            row['release_year']+0.3,
            row['log_play_count']-0.1,
            row['track_name'],
            fontsize=12,
            ha='left',
            color=row['text_color']
        )
    )
adjust_text(texts,arrowprops=dict(arrowstyle='->', color='red'))


ax.axvline(x=x_mid, color='#c2d6f3', linestyle='--', linewidth=1)
ax.axhline(y=y_mid, color='#c2d6f3', linestyle='--', linewidth=1)
ax.set_xlim(1985,2025)
ax.set_ylim(8,18)

ax.set_title('Hacken Lee X HKPhil Concert 2023 Rundown', fontsize=16)
ax.set_xlabel('Release Year', fontsize=12)
ax.set_ylabel('Log-Scaled Popularity', fontsize=12)
ax.legend(title="Record Company", loc='upper right')
plt.text(0.85,
         0.77,
         "a watermark by wend1k3",
         transform=plt.gca().transAxes,
            ha='center',  
            va='center',  
            alpha=1,
                fontdict=dict(
                    fontsize=11,
                    color='#ecacbd',
                    family=
                    "Heiti TC",  
                    weight=
                    'normal', 
                )  )
plt.show()


Here are the three quadrant charts corresponding to three setlist.
![](img/23HKquadrant.png)
![](img/24GZquadrant.png)
![](img/25OMquadrant.png)

### 4.2 Bar plot 

## 5. Discussion